In [3]:
import sys
import os
from pathlib import Path
import tensorflow as tf
import numpy as np

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.architecture as ARCH
import SnowDepth.optimal_features as OF

In [4]:
# Assign seed
seed = 18

# Path to TIFF files
data_dir = ROOT/"data"/"tif_files"

# Select holdout AOI
holdout_aoi="ID_BS"

# Select max amount of features to select from FF algos
top_k = 10

# Load dataframe
df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)

dev_df  = df[df['aoi_name'] != holdout_aoi].copy()
hold_df = df[df['aoi_name'] == holdout_aoi].copy()


# Directory for writing H5 files
h5_dir = ROOT/"data"/"h5_dir"

In [5]:
# Run FF algorithms
ff_algos = OF.optimal_feature_sets(dev_df, top_k=10, n_per_aoi=10000)

# Write one H5 per feature-set
for name, feats in ff_algos.items():
    out_dir = h5_dir / name
    out_dir.mkdir(parents=True, exist_ok=True)

    h5_file = out_dir / f"data_{name}.h5"

    if not h5_file.exists():
        DL.build_h5(
            data_dir=str(data_dir),
            out_dir=str(out_dir),
            write_mask=True,
            upper_threshold=3.0,
            selected_features=feats,
            out_name=f"data_{name}.h5"
        )
    else:
        print(f"Using existing H5: {h5_file}")

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Gaussian kernel for the outcomes.
HSIC selected: ['IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Beta_ratio', 'Slope', 'LIA', 'Gamma_RTC_ratio', 'Beta_VH', 'sin_Aspect']
PCC selected: ['IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Elevation', 'Beta_VH', 'Slope', 'LIA']
MI selected): ['IAFE', 'Elevation', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Gamma_RTC_sum', 'cos_Aspect', 'Beta_VH', 'Slope', 'Gamma_ratio']
Wrote HDF5 with 6 AOI(s) at c:\Users\mathi\Documents\Paper2\ML_SD\data\h5_dir\HSIC\data_HSIC.h5
features: ['IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Beta_ratio', 'Slope', 'LIA', 'Gamma_RTC_ratio', 'Beta_VH', 'sin_Aspect']
Wrote HDF5 with 6 AOI(s) at c:\Users\mathi\Documents\Paper2\ML_SD\data\h5_dir\PCC\data_PCC.h5
features: ['IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Elevation', 'Beta_VH', 'Slope', 'LIA']
Wrote HDF5 with 6 AOI(s) at c:\Users\mathi\Documents\Paper2\ML_SD\da

In [ ]:
# H5 paths per FF set
h5_path_HSIC = h5_dir / "HSIC" / "data_HSIC.h5"
h5_path_PCC  = h5_dir / "PCC"  / "data_PCC.h5"
h5_path_MI   = h5_dir / "MI"   / "data_MI.h5"

# Helpers
def zscore_from_train(x_train, *xs, eps=1e-6):
    mean = x_train.mean(axis=(0, 1, 2), keepdims=True)
    std  = x_train.std(axis=(0, 1, 2), keepdims=True) + eps
    return tuple((x - mean) / std for x in (x_train,) + xs)

def fill_nan_and_mask(y):
    # y: (N, H, W, 1)
    mask = (~np.isnan(y))[..., 0].astype("float32")   # (N, H, W)
    y_filled = np.where(np.isnan(y), 0.0, y).astype("float32")
    return y_filled, mask

# HSIC
(X_train_HSIC, y_train_HSIC), (X_val_HSIC, y_val_HSIC), (X_hold_HSIC, y_hold_HSIC) = DS.unet_split(
    h5_path=str(h5_path_HSIC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.30,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("HSIC Shapes:",
      "X_train", X_train_HSIC.shape, "y_train", y_train_HSIC.shape,
      "X_val",   X_val_HSIC.shape,   "y_val",   y_val_HSIC.shape,
      "X_hold",  X_hold_HSIC.shape,  "y_hold",  y_hold_HSIC.shape)

(X_train_n_HSIC, X_val_n_HSIC, X_hold_n_HSIC) = zscore_from_train(X_train_HSIC, X_val_HSIC, X_hold_HSIC)
y_train_f_HSIC, w_train_HSIC = fill_nan_and_mask(y_train_HSIC)
y_val_f_HSIC,   w_val_HSIC   = fill_nan_and_mask(y_val_HSIC)
y_hold_f_HSIC,  w_hold_HSIC  = fill_nan_and_mask(y_hold_HSIC)
X_train_n_HSIC = X_train_n_HSIC.astype("float32"); X_val_n_HSIC = X_val_n_HSIC.astype("float32"); X_hold_n_HSIC = X_hold_n_HSIC.astype("float32")

# PCC
(X_train_PCC, y_train_PCC), (X_val_PCC, y_val_PCC), (X_hold_PCC, y_hold_PCC) = DS.unet_split(
    h5_path=str(h5_path_PCC),
    holdout_aoi=holdout_aoi,
    val_fraction=0.30,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("PCC  Shapes:",
      "X_train", X_train_PCC.shape, "y_train", y_train_PCC.shape,
      "X_val",   X_val_PCC.shape,   "y_val",   y_val_PCC.shape,
      "X_hold",  X_hold_PCC.shape,  "y_hold",  y_hold_PCC.shape)

(X_train_n_PCC, X_val_n_PCC, X_hold_n_PCC) = zscore_from_train(X_train_PCC, X_val_PCC, X_hold_PCC)
y_train_f_PCC, w_train_PCC = fill_nan_and_mask(y_train_PCC)
y_val_f_PCC,   w_val_PCC   = fill_nan_and_mask(y_val_PCC)
y_hold_f_PCC,  w_hold_PCC  = fill_nan_and_mask(y_hold_PCC)
X_train_n_PCC = X_train_n_PCC.astype("float32"); X_val_n_PCC = X_val_n_PCC.astype("float32"); X_hold_n_PCC = X_hold_n_PCC.astype("float32")

# MI
(X_train_MI, y_train_MI), (X_val_MI, y_val_MI), (X_hold_MI, y_hold_MI) = DS.unet_split(
    h5_path=str(h5_path_MI),
    holdout_aoi=holdout_aoi,
    val_fraction=0.30,
    seed=seed,
    patch_size=128,
    stride=64,
    min_valid_frac=0.80
)
print("MI   Shapes:",
      "X_train", X_train_MI.shape, "y_train", y_train_MI.shape,
      "X_val",   X_val_MI.shape,   "y_val",   y_val_MI.shape,
      "X_hold",  X_hold_MI.shape,  "y_hold",  y_hold_MI.shape)

(X_train_n_MI, X_val_n_MI, X_hold_n_MI) = zscore_from_train(X_train_MI, X_val_MI, X_hold_MI)
y_train_f_MI, w_train_MI = fill_nan_and_mask(y_train_MI)
y_val_f_MI,   w_val_MI   = fill_nan_and_mask(y_val_MI)
y_hold_f_MI,  w_hold_MI  = fill_nan_and_mask(y_hold_MI)
X_train_n_MI = X_train_n_MI.astype("float32"); X_val_n_MI = X_val_n_MI.astype("float32"); X_hold_n_MI = X_hold_n_MI.astype("float32")


Shapes:
  X_train: (212, 128, 128, 7)  y_train: (212, 128, 128, 1)
  X_val:   (43, 128, 128, 7)    y_val:   (43, 128, 128, 1)
  X_hold:  (362, 128, 128, 7)   y_hold:  (362, 128, 128, 1)


In [ ]:
# ---- HSIC UNet training ----
model_HSIC = ARCH.unet(input_shape=X_train_n_HSIC.shape[1:], base_filters=32)
LR = 1e-3
model_HSIC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
os.makedirs("UNet_weights", exist_ok=True)
ckpt_HSIC = "UNet_weights/unet_best_HSIC.weights.h5"
callbacks_HSIC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_HSIC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_HSIC = model_HSIC.fit(
    X_train_n_HSIC, y_train_f_HSIC,
    sample_weight=w_train_HSIC,
    validation_data=(X_val_n_HSIC, y_val_f_HSIC, w_val_HSIC),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_HSIC,
    verbose=1,
)

# PCC UNet training
model_PCC = ARCH.unet(input_shape=X_train_n_PCC.shape[1:], base_filters=32)
model_PCC.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
ckpt_PCC = "UNet_weights/unet_best_PCC.weights.h5"
callbacks_PCC = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_PCC, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_PCC = model_PCC.fit(
    X_train_n_PCC, y_train_f_PCC,
    sample_weight=w_train_PCC,
    validation_data=(X_val_n_PCC, y_val_f_PCC, w_val_PCC),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_PCC,
    verbose=1,
)

# MI UNet training
model_MI = ARCH.unet(input_shape=X_train_n_MI.shape[1:], base_filters=32)
model_MI.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)
ckpt_MI = "UNet_weights/unet_best_MI.weights.h5"
callbacks_MI = [
    tf.keras.callbacks.ModelCheckpoint(ckpt_MI, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5),
]
hist_MI = model_MI.fit(
    X_train_n_MI, y_train_f_MI,
    sample_weight=w_train_MI,
    validation_data=(X_val_n_MI, y_val_f_MI, w_val_MI),
    epochs=50,
    batch_size=4,
    callbacks=callbacks_MI,
    verbose=1,
)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - MAE: 0.9361 - loss: 0.5546
Epoch 1: val_loss improved from inf to 156.48790, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 364ms/step - MAE: 0.9309 - loss: 0.5502 - val_MAE: 160.2921 - val_loss: 156.4879 - learning_rate: 0.0010
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - MAE: 0.4539 - loss: 0.1459
Epoch 2: val_loss improved from 156.48790 to 4.71764, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 19s 350ms/step - MAE: 0.4536 - loss: 0.1457 - val_MAE: 5.2918 - val_loss: 4.7176 - learning_rate: 0.0010
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - MAE: 0.4295 - loss: 0.1348
Epoch 3: val_loss improved from 4.71764 to 0.53150, saving model to weights/unet_best.weights.h5
53/53 ━━━━━━━━━━━━━━━━━━━━ 18s 339ms/step - MAE: 0.4294 - loss: 0.1348 - val_MAE: 0.9598 - val_loss: 0.5315 - learning_rate: 0.0010
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - MAE: 0.4201 

In [9]:
y_pred_hold = model.predict(X_hold_n)

def masked_rmse_mae(y_true, y_pred, w, eps=1e-6):
    # y_*: (N,H,W,1), w: (N,H,W)
    diff = (y_true[...,0] - y_pred[...,0])
    mae = (np.abs(diff) * w).sum() / (w.sum() + eps)
    rmse = np.sqrt(((diff**2) * w).sum() / (w.sum() + eps))
    return rmse, mae

rmse, mae = masked_rmse_mae(y_hold_f, y_pred_hold, w_hold)
print(f"Hold-out (masked) — RMSE: {rmse:.4f}, MAE: {mae:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 683ms/step
Hold-out (masked) — RMSE: 0.4643, MAE: 0.3598
